# to connect with spotify

In [ ]:
# # First cell - update your authentication code

# import spotipy
# from spotipy.oauth2 import SpotifyOAuth
# from dotenv import load_dotenv
# import os

# load_dotenv()

# CLIENT_ID     = os.getenv("CLIENT_ID")
# CLIENT_SECRET = os.getenv("CLIENT_SECRET")
# REDIRECT_URI  = os.getenv("REDIRECT_URI", "http://127.0.0.1:8000/error")

# if not CLIENT_ID or not CLIENT_SECRET:
#     raise RuntimeError("Missing CLIENT_ID or CLIENT_SECRET in environment")

# # Create auth manager with explicit cache path
# auth_manager = SpotifyOAuth(
#     client_id=CLIENT_ID,
#     client_secret=CLIENT_SECRET,
#     redirect_uri=REDIRECT_URI,
#     scope="user-library-read playlist-modify-public user-top-read user-read-recently-played user-read-private user-read-email",
#     cache_path=".spotify_cache"  # Add this line
# )

# # Create client with auth manager
# sp = spotipy.Spotify(auth_manager=auth_manager)

# # Force a token refresh
# sp.auth_manager.get_access_token(as_dict=False)

# # Check authentication status
# user = sp.current_user()
# print(f"✓ Authentication successful - logged in as: {user['display_name']}")

In [ ]:
# def signin():
#     """
#     Sign in to Spotify using the Spotipy library.
#     This function will open a web browser for the user to log in.
#     """
#     auth_manager = SpotifyOAuth(
#     client_id=CLIENT_ID,
#     client_secret=CLIENT_SECRET,
#     redirect_uri=REDIRECT_URI,
#     scope="user-library-read playlist-modify-public user-top-read user-read-recently-played user-read-private user-read-email",
#     cache_path=".spotify_cache"  # Add this line
# )

#     # Create client with auth manager
#     sp = spotipy.Spotify(auth_manager=auth_manager)

#     # Force a token refresh
#     sp.auth_manager.get_access_token(as_dict=False)

#     # Check authentication status
#     user = sp.current_user()
#     print(f"✓ Authentication successful - logged in as: {user['display_name']}")

#     return user,sp

In [1]:
from src.onboard import signin, get_sql_connection
conn = get_sql_connection()
user,sp = signin(conn)

✓ Connected to SQL Server
✓ Logged in as: Ankit Sneh
✓ User data handled successfully


In [ ]:
# import spotipy
# from spotipy.oauth2 import SpotifyOAuth
# from dotenv import load_dotenv
# import os

# load_dotenv()

# CLIENT_ID     = os.getenv("CLIENT_ID")
# CLIENT_SECRET = os.getenv("CLIENT_SECRET")
# REDIRECT_URI  = os.getenv("REDIRECT_URI", "http://127.0.0.1:8000/error")

# if not CLIENT_ID or not CLIENT_SECRET:
#     raise RuntimeError("Missing CLIENT_ID or CLIENT_SECRET in environment")

# user,sp = signin()

TypeError: signin() missing 1 required positional argument: 'conn'

## top songs


In [2]:
from src.onboard import get_top_tracks
# top_tracks = sp.current_user_top_tracks(limit=20, time_range='short_term')
# for track in top_tracks['items']:
    # print(track['name'], "-", track['artists'][0]['name'])
hello = get_top_tracks(sp,conn)


## all liked tracks

In [3]:
from src.onboard import get_liked_tracks
liked = get_liked_tracks(sp,conn)

In [ ]:
# new code cell: fetch all liked songs

limit = 50
offset = 0
liked_songs = []

while True:
    resp = sp.current_user_saved_tracks(limit=limit, offset=offset)
    items = resp.get('items', [])
    if not items:
        break
    liked_songs.extend(items)
    offset += len(items)



# Now print out track name – artist(s)
for item in liked_songs:
    track = item['track']
    artists = ", ".join(a['name'] for a in track['artists'])
    print(f"{track['name']}  –  {artists}")

# new code cell: export liked songs + metadata to JSON



## to get the meta data

In [ ]:
import json

# Helper to selectively remove unnecessary fields
def clean_track_data(track_item):
    # Keep the added_at date and clean the track data
    cleaned = {
        "added_at": track_item.get("added_at"),
        "track": {
            "id": track_item["track"]["id"],
            "name": track_item["track"]["name"],
            "popularity": track_item["track"]["popularity"],
            "duration_ms": track_item["track"]["duration_ms"],
            "explicit": track_item["track"]["explicit"],
            "artists": [{
                "id": artist["id"],
                "name": artist["name"]
            } for artist in track_item["track"]["artists"]],
            "album": {
                "id": track_item["track"]["album"]["id"],
                "name": track_item["track"]["album"]["name"],
                "release_date": track_item["track"]["album"]["release_date"],
                "album_type": track_item["track"]["album"]["album_type"],
            }
        }
    }
    return cleaned

# Process the liked songs, cleaning unnecessary data
clean_liked_songs = [clean_track_data(item) for item in liked_songs]

# Export the data
with open('liked_songs_clean.json', 'w') as f:
    json.dump(clean_liked_songs, f, indent=2)

print(f"Successfully cleaned and saved {len(clean_liked_songs)} tracks")
print("Data saved to liked_songs_clean.json")

KeyError: 'track'

## scraper


In [ ]:
import json
import time
import os
from utils.web_scraper import get_song_details, save_song_lyrics

# Load the liked songs JSON file
with open('liked_songs_clean.json', 'r') as file:
    liked_songs = json.load(file)

# Extract artist and song name for each track
songs_list = []
for item in liked_songs:
    track = item['track']
    song_name = track['name']
    # Get first artist (some tracks have multiple artists)
    first_artist = track['artists'][0]['name']
    
    songs_list.append((first_artist, song_name))

# Print the list of extracted songs
print(f"Found {len(songs_list)} tracks:")
for i, (artist, song) in enumerate(songs_list[:10]):
    print(f"{i+1}. {artist} - {song}")
print("...")

# Create base directory for lyrics
os.makedirs("song_details", exist_ok=True)

# Process songs in batches with controlled rate limiting
BATCH_SIZE = 5  # Number of songs to process in one run
start_idx = 0  # Starting index (change this to continue from where you left off)
end_idx = min(start_idx + BATCH_SIZE, len(songs_list))

print(f"\nProcessing songs {start_idx+1} to {end_idx} (out of {len(songs_list)})")

successful = 0
failed = 0

for i, (artist, song) in enumerate(songs_list[start_idx:end_idx], start=start_idx+1):
    print(f"\nProcessing {i}/{len(songs_list)}: {artist} - {song}")
    
    try:
        # First get song details
        song_details = get_song_details(artist, song)
        
        # Check if lyrics were found
        if song_details and "lyrics" in song_details and song_details["lyrics"] and not song_details["lyrics"].startswith("Could not find"):
            print(f"✅ Lyrics found ({len(song_details['lyrics'])} chars)")
            
            # Save the details using the correct function signature
            # Assuming your updated save_song_lyrics takes artist and song separately:
            save_song_lyrics(song_details)
            successful += 1
        else:
            print(f"❌ No lyrics found")
            failed += 1
            
        # Add delay between requests (2-4 seconds)
        delay = 2 + (i % 3)  # Vary delay slightly
        print(f"Waiting {delay} seconds before next song...")
        time.sleep(delay)
        
    except Exception as e:
        print(f"❌ Error processing {artist} - {song}: {str(e)}")
        failed += 1
        time.sleep(3)  # Longer delay after errors

print(f"\n--- Summary ---")

print(f"Processed: {end_idx - start_idx} songs")
print(f"Success: {successful}")
print(f"Failed: {failed}")
print(f"\nTo continue with the next batch, set start_idx = {end_idx}")

## search


In [ ]:
def search_songs(query, limit=10):
    """
    Search for songs on Spotify and display results.
    
    Parameters:
        - query: Search string for finding tracks
        - limit: Maximum number of results to return (max 50)
    
    Returns:
        List of track dictionaries from the search results
    """
    # Perform the search using the Spotify client
    results = sp.search(
        q=query,
        limit=limit,
        type="track",
        market="from_token"  # Uses the user's market from their token
    )
    
    # Extract the tracks from the results
    tracks = results['tracks']['items']
    
    # Display the results
    print(f"\n--- Found {len(tracks)} results for '{query}' ---\n")
    
    for i, track in enumerate(tracks, 1):
        artists = ", ".join([artist['name'] for artist in track['artists']])
        album = track['album']['name']
        duration_ms = track['duration_ms']
        duration = f"{duration_ms // 60000}:{(duration_ms % 60000) // 1000:02d}"  # Convert to minutes:seconds
        
        print(f"{i}. {track['name']} - {artists}")
        print(f"   Album: {album} | Duration: {duration} | Popularity: {track['popularity']}/100")
        print(f"   ID: {track['id']}")
        print()
    
    return tracks

# Example usage
if __name__ == "__main__":
    query = input("Enter a search term: ")
    tracks = search_songs(query)
    print(f"\nFound {len(tracks)} tracks matching '{query}'")
    # Optionally select a track to create a playlist with
    # if tracks:
    #     selection = input("\nEnter track number to create a playlist with this song (or press Enter to skip): ")
    #     if selection.isdigit() and 1 <= int(selection) <= len(tracks):
    #         selected_track = tracks[int(selection) - 1]
    #         playlist_name = f"Playlist with {selected_track['name']}"
            
            # Create a playlist with the selected track
            # create_playlist_from_tracks(
            #     track_ids=[selected_track['id']],
            #     playlist_name=playlist_name,
            #     playlist_description=f"Playlist featuring {selected_track['name']} by {selected_track['artists'][0]['name']}"
            # )

## Getting recommend and new songs (not working)

In [ ]:
import json
import time
from urllib.parse import urlencode
import spotipy
import random
from IPython.display import display, HTML

def create_playlist_from_tracks(track_ids, playlist_name=None, playlist_description=None):
    """
    Create a new Spotify playlist from a list of track IDs
    
    Parameters:
        track_ids (list): List of Spotify track IDs to add to the playlist
        playlist_name (str): Name for the new playlist (default: "Recommended Tracks YYYY-MM-DD")
        playlist_description (str): Description for the new playlist
        
    Returns:
        str: URL to the created playlist
    """
    if not track_ids:
        print("❌ No tracks provided to create playlist")
        return None
    rand = random.randint(1, 1000)
        
    # Use default name if none provided
    if not playlist_name:
        playlist_name = f"SpotAIfy-{rand}-{time.strftime('%Y-%m-%d')}"
        
    # Use default description if none provided
    if not playlist_description:
        playlist_description = f"Playlist created on {time.strftime('%Y-%m-%d')} with recommended tracks"
    
    try:
        # Get current user ID
        user_id = sp.current_user()['id']
        
        # Create an empty playlist
        print(f"Creating new playlist: '{playlist_name}'")
        created_playlist = sp.user_playlist_create(
            user=user_id,
            name=playlist_name,
            public=True,
            description=playlist_description
        )
        
        # Get the playlist ID
        playlist_id = created_playlist['id']
        
        # Add tracks in batches (Spotify has a limit of 100 per request)
        batch_size = 100
        for i in range(0, len(track_ids), batch_size):
            batch = track_ids[i:i+batch_size]
            sp.playlist_add_items(playlist_id=playlist_id, items=batch)
            print(f"Added tracks {i+1}-{i+len(batch)} to playlist")
        
        # Get the playlist URL
        playlist_url = created_playlist['external_urls']['spotify']
        
        print(f"✅ Successfully created playlist with {len(track_ids)} tracks!")
        print(f"Playlist URL: {playlist_url}")
        
        # Display clickable link (works in Jupyter)
        display(HTML(f'<a href="{playlist_url}" target="_blank">Open playlist in Spotify</a>'))
        
        return playlist_url
        
    except Exception as e:
        print(f"❌ Error creating playlist: {str(e)}")
        return None


In [ ]:
import json
import time
import random
from IPython.display import display, HTML

def create_playlist_with_recommendations():
    """Get recommendations using Spotipy's built-in function and create a playlist"""
    
    print("Getting recommendations...")
    
    try:
        # Load liked songs to get seed tracks
        with open('liked_songs_clean.json', 'r') as f:
            liked_songs = json.load(f)
            
        # Get 2 random tracks as seeds
        if len(liked_songs) > 2:
            random_tracks = random.sample(liked_songs, 2)
            seed_track_ids = [track['track']['id'] for track in random_tracks]
            print(f"Using seed tracks: {', '.join([t['track']['name'] for t in random_tracks])}")
        else:
            seed_track_ids = []
        
        # Add a specific artist as seed
        seed_artist_id = "1Xyo4u8uXC1ZmMpatF05PJ"  # The Weeknd
        print(f"Using seed artist: The Weeknd")
        
        # NOTE: The total number of seeds (tracks + artists + genres) must be ≤ 5
        # Call the recommendations function properly with list parameters
        recommendations = sp.recommendations(
            seed_artists=[seed_artist_id],  # Must be a list
            seed_tracks=seed_track_ids,     # Already a list
            limit=25
        )
        
        # Extract track IDs from the response
        if recommendations and 'tracks' in recommendations:
            track_ids = [track['id'] for track in recommendations['tracks']]
            print(f"✅ Got {len(track_ids)} recommendations")
            
            # Create playlist
            create_playlist_from_tracks(
                track_ids=track_ids,
                playlist_name=f"SpotAIfy-{random.randint(1, 1000)}-{time.strftime('%Y-%m-%d')}",
                playlist_description="Spotify recommendations based on your music taste"
            )
        else:
            raise Exception("No recommendations returned")
            
    except Exception as e:
        print(f"❌ Error getting recommendations: {str(e)}")
        
        # Fallback: Use top tracks instead
        print("\nFallback: Using your top tracks instead")
        try:
            top_tracks = sp.current_user_top_tracks(limit=25, time_range='medium_term')
            top_track_ids = [track['id'] for track in top_tracks['items']]
            
            create_playlist_from_tracks(
                track_ids=top_track_ids,
                playlist_name=f"SpotAIfy-top-{random.randint(1, 1000)}-{time.strftime('%Y-%m-%d')}",
                playlist_description="A collection of your most played songs"
            )
        except Exception as e2:
            print(f"❌ Error with fallback: {str(e2)}")

# Run the function
create_playlist_with_recommendations()

## Recommender new


In [59]:
from llm_axe import OllamaChat, Agent, OnlineAgent
import re, json

def get_recommendations(prompt):
    prompt2 = f"""
You are a music recommendation agent.

Your task is to recommend songs based on the user's emotional state or request. You must follow these rules strictly:

1.  Recommend a MAXIMUM of 20 songs.
2.  Only suggest songs that are **not overrated**.
3.  Respond in **valid JSON format only**, with NO extra text, no commentary, and no explanations.
4.  Each song must follow this format:
   {{
       "name": "Name of the song",
       "artist": "Main artist of the song"
   }}
5. If the user request is unrelated to music or songs, respond with this exact text (without JSON):  
   `"I CAN'T HELP YOU WITH THAT"`

MAIN TASK:  
Determine the user's mood or intent based on this input, and return an appropriate list of songs in JSON:

USER INPUT: {prompt}
"""
    llm = OllamaChat(model="gemma3:4b-it-qat")
    searcher = OnlineAgent(llm)
    resp_raw = searcher.search(prompt2)
    print("Raw response:", resp_raw)
    # coerce into a string
    resp = resp_raw if isinstance(resp_raw, str) else getattr(resp_raw, "text", str(resp_raw))

    # now regex on resp
    match = re.search(r"json\s*(\[\s*{.*?}\s*])\s*", resp, re.DOTALL)
    if match:
        js = match.group(1)
        try:
            data = json.loads(js)
            with open("recommendations.json", "w") as f:
                json.dump(data, f, indent=2)
            print("Extracted JSON:", data)
        except json.JSONDecodeError as e:
            print("❌ JSON decode error:", e)
    else:
        print("❌ No JSON found in response:\n", resp)

    return resp

# test it


In [58]:
resp = get_recommendations("am feeling emotionally available and sad at the same time, suggest me smth which can make me feel good")
print(resp)

Raw response: Based on information from the internet, 
```json
[
 {
  "name": "Rivers and Roads",
  "artist": "The Head and the Heart"
 },
 {
  "name": "Hallelujah",
  "artist": "Jeff Buckley"
 },
 {
  "name": "Landslide",
  "artist": "Fleetwood Mac"
 },
 {
  "name": "Skinny Love",
  "artist": "Bon Iver"
 },
 {
  "name": "The Scientist",
  "artist": "Coldplay"
 }
]
```
Extracted JSON: [{'name': 'Rivers and Roads', 'artist': 'The Head and the Heart'}, {'name': 'Hallelujah', 'artist': 'Jeff Buckley'}, {'name': 'Landslide', 'artist': 'Fleetwood Mac'}, {'name': 'Skinny Love', 'artist': 'Bon Iver'}, {'name': 'The Scientist', 'artist': 'Coldplay'}]
Based on information from the internet, 
```json
[
 {
  "name": "Rivers and Roads",
  "artist": "The Head and the Heart"
 },
 {
  "name": "Hallelujah",
  "artist": "Jeff Buckley"
 },
 {
  "name": "Landslide",
  "artist": "Fleetwood Mac"
 },
 {
  "name": "Skinny Love",
  "artist": "Bon Iver"
 },
 {
  "name": "The Scientist",
  "artist": "Coldplay"


In [ ]:
# print(resp)
import re
import json


Extracted JSON: [{'name': 'Someone You Loved', 'artist': 'Lewis Capaldi'}, {'name': 'Say Nothing', 'artist': 'Lewis Capaldi'}, {'name': 'All Too Well (10 Minute Version)', 'artist': 'Taylor Swift'}, {'name': 'Fix You', 'artist': 'Coldplay'}, {'name': 'Hallelujah', 'artist': 'Jeff Buckley'}, {'name': 'Liability', 'artist': 'Taylor Swift'}, {'name': 'The Scientist', 'artist': 'Coldplay'}, {'name': 'Nothing Compares 2 U', 'artist': "Sinead O'Connor"}, {'name': 'Back to Black', 'artist': 'Amy Winehouse'}, {'name': 'Skinny Love', 'artist': 'Bon Iver'}, {'name': 'I Will Survive', 'artist': 'Gloria Gaynor'}, {'name': 'Heartbreak Hotel', 'artist': 'Elvis Presley'}, {'name': 'Without You', 'artist': 'Harry Nilsson'}, {'name': 'River', 'artist': 'Metallica'}, {'name': 'A Case of You', 'artist': 'Joni Mitchell'}, {'name': 'The Sound of Silence', 'artist': 'Simon & Garfunkel'}, {'name': 'Fast Car', 'artist': 'Tracy Chapman'}, {'name': 'Teardrops on My Guitar', 'artist': 'Taylor Swift'}, {'name': '

## making a playlist from searching 

In [60]:
import json

def fetch_recommendation_ids(recs, sp):
    """
    For each recommendation dict {'name':…, 'artist':…},
    search Spotify and return the first track ID (or None).
    """
    ids = []
    for rec in recs:
        query = f"{rec['name']} {rec['artist']}"
        res = sp.search(q=query, limit=1, type="track", market="from_token")
        items = res.get("tracks", {}).get("items", [])
        if items:
            ids.append(items[0]["id"])
        else:
            ids.append(None)
    return ids

with open("recommendations.json", "r") as f:
    recs = json.load(f)

track_ids = fetch_recommendation_ids(recs, sp)
print(track_ids)

['4X6aeSW2bhlbm7YyEstj6Y', '3pRaLNL3b8x5uBOcsgvdqM', '3X3mfRMRaPJRiIMDMQtSkd', '3B3eOgLJSqPEA0RfboIQVM', '75JFxkI2RXiU7L9VXzMkle']


In [61]:
import json
import time
from urllib.parse import urlencode
import spotipy
import random
from IPython.display import display, HTML

def create_playlist_from_tracks(track_ids, playlist_name=None, playlist_description=None):
    """
    Create a new Spotify playlist from a list of track IDs
    
    Parameters:
        track_ids (list): List of Spotify track IDs to add to the playlist
        playlist_name (str): Name for the new playlist (default: "Recommended Tracks YYYY-MM-DD")
        playlist_description (str): Description for the new playlist
        
    Returns:
        str: URL to the created playlist
    """
    if not track_ids:
        print("❌ No tracks provided to create playlist")
        return None
    rand = random.randint(1, 1000)
        
    # Use default name if none provided
    if not playlist_name:
        playlist_name = f"SpotAIfy-{rand}-{time.strftime('%Y-%m-%d')}"
        
    # Use default description if none provided
    if not playlist_description:
        playlist_description = f"Playlist created on {time.strftime('%Y-%m-%d')} with recommended tracks"
    
    try:
        # Get current user ID
        user_id = sp.current_user()['id']
        
        # Create an empty playlist
        print(f"Creating new playlist: '{playlist_name}'")
        created_playlist = sp.user_playlist_create(
            user=user_id,
            name=playlist_name,
            public=True,
            description=playlist_description
        )
        
        # Get the playlist ID
        playlist_id = created_playlist['id']
        
        # Add tracks in batches (Spotify has a limit of 100 per request)
        batch_size = 100
        for i in range(0, len(track_ids), batch_size):
            batch = track_ids[i:i+batch_size]
            sp.playlist_add_items(playlist_id=playlist_id, items=batch)
            print(f"Added tracks {i+1}-{i+len(batch)} to playlist")
        
        # Get the playlist URL
        playlist_url = created_playlist['external_urls']['spotify']
        
        print(f"✅ Successfully created playlist with {len(track_ids)} tracks!")
        print(f"Playlist URL: {playlist_url}")
        
        # Display clickable link (works in Jupyter)
        display(HTML(f'<a href="{playlist_url}" target="_blank">Open playlist in Spotify</a>'))
        
        return playlist_url
        
    except Exception as e:
        print(f"❌ Error creating playlist: {str(e)}")
        return None

create_playlist_from_tracks(track_ids)

Creating new playlist: 'SpotAIfy-4-2025-05-11'
Added tracks 1-5 to playlist
✅ Successfully created playlist with 5 tracks!
Playlist URL: https://open.spotify.com/playlist/0DHawOIkYj3yVBHCrXxdyF


'https://open.spotify.com/playlist/0DHawOIkYj3yVBHCrXxdyF'